In [1]:
from __future__ import print_function
from tesserocr import PyTessBaseAPI, RIL, iterate_level, PT, OEM
from PIL import Image
import cv2
import numpy as np

import time

In [2]:
IMAGEPATH = '../datafiles/tb-doc-1.jpg'

In [3]:
def draw_rects(img, x,y,x2,y2, block_type, conf):
    """
   ?????????????
   :param img:
   :param rects:
   :return:
    """
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.rectangle(img, (x, y), (x2, y2), (0, 0, 255), 2)
    cv2.imshow('Image',img)
    cv2.waitKey(5)
   #cv2.imwrite("image" + str(block_type) + ".jpg", img)
   #plt.imsave('./final_image.jpg',img)
    cv2.putText(img, str(conf), (x, y), font, 1, (0, 255, 0), 2, cv2.LINE_AA)

In [25]:
def get_table_row_tags(image):
    with PyTessBaseAPI(path="C:\\Tesseract\\Tesseract-OCR-v5\\tessdata") as api:
        api.SetImage(image)
        api.Recognize()
        
        #GET PARA TEXT
        ri = api.GetIterator()
        level = RIL.BLOCK
        
        rows_bbox = []
        for r in iterate_level(ri, level):
            block_type = r.BlockType()
            if (block_type == PT.HORZ_LINE):
                tmp = {}
                tmp['bbox'] = r.BoundingBox(level)
                rows_bbox.append(tmp)
        
        #GET PARA TEXT
        ri = api.GetIterator()
        level = RIL.WORD
        
        for r in iterate_level(ri, level):
            word = r.GetUTF8Text(level)
            curr_bbox = r.BoundingBox(level)
            print(curr_bbox, word)
            for i in range(len(rows_bbox)):
                if (curr_bbox[1] <= rows_bbox[i]['bbox'][1]):
                    if 'text' not in list(rows_bbox[i].keys()):
                        rows_bbox[i]['text'] = word
                    else:
                        rows_bbox[i]['text'] += " "+word
                    break
            
        table_text = "<table>\n<th>\n"+rows_bbox[0]['text']+'\n</th>\n'
        for i in rows_bbox[1:]:
            table_text = "<tr>\n"+i[0]['text']+'\n</tr>\n'
        table_text += "</table>/n"
        
        return table_text

In [24]:
start = time.time()
with PyTessBaseAPI(path="C:\\Tesseract\\Tesseract-OCR-v5\\tessdata") as api:
    api.SetImageFile(IMAGEPATH)
    
    api.Recognize()
    print("time:\t", time.time() - start)
    image = Image.open(IMAGEPATH)
    image_np = np.array(image)

    #GET TABLE IMAGE
    ri = api.GetIterator()
    level = RIL.BLOCK

    for r in iterate_level(ri, level):
        block_type = r.BlockType()
        
        #print(block_type, block_type == PT.TABLE)
        if (block_type == PT.TABLE):
            table_image = r.GetImage(level, 5, image)
            text_rows = get_table_row_tags(table_image[0])            
                

time:	 1.7301514148712158
(4, 18, 500, 20)  
(4, 40, 500, 42)  
(4, 61, 500, 63)  
(4, 83, 500, 85)  
(4, 105, 500, 107)  
(4, 126, 500, 128)  
(4, 148, 500, 150)  
(4, 169, 500, 171)  
(4, 191, 500, 193)  
(4, 213, 500, 215)  
(4, 234, 500, 236)  
(4, 256, 500, 258)  
(4, 0, 6, 279)  
(105, 0, 107, 279)  
(212, 0, 214, 279)  
(360, 0, 362, 279)  
(438, 5, 440, 286)  
(17, 5, 50, 15) Speed
(55, 5, 89, 15) (mph)
(0, 0, 500, 308) _|
(116, 5, 150, 13) Driver
(225, 5, 243, 13) ar
(372, 5, 408, 15) Engine
(0, 0, 500, 308) |
(449, 5, 474, 13) Date
(16, 27, 57, 35) 407.447
(116, 27, 140, 37) Craig
(145, 27, 193, 35) Breediove
(0, 0, 500, 308) |
(225, 26, 264, 36) Snintot
(267, 26, 306, 34) Ameria
(372, 26, 405, 34) GEM?
(449, 26, 482, 36) 5/68
(16, 49, 57, 57) 413.159
(115, 49, 136, 57) Tum
(140, 49, 169, 57) Green
(224, 48, 268, 58) Wirgtoot
(273, 48, 310, 58) Express
(371, 48, 408, 56) wens
(0, 0, 500, 308) |
(449, 48, 489, 58) auayea
(16, 70, 51, 78) 434.22
(115, 70, 130, 78) Are
(133, 70,